EDA com objetivo de entender profundamente o dataset, suas características e peculiaridades para que possa ser planejada a amostragem de forma inteligente e eficaz.

In [ ]:
#Dependências
import pandas as pd
import seaborn as sns
from datasets import load_dataset
import os

In [ ]:
# Carregar o dataset
dataset = load_dataset("eliasjacob/simulados_enem")

  Using cached datasets-4.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached hf_xet-1.1.10-cp37-abi3-macosx_11_0_arm64.whl.metadata (4.7 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached datasets-4.1.0-py3-none-any.whl (503 kB)
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.6/562.6 kB 22.0 MB/s 

README.md:   0%|          | 0.00/428 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7357 [00:00<?, ? examples/s]

In [24]:
#Convertendo o dataset para pandas dataframe para manipulação mais fácil
df = dataset['train'].to_pandas()

In [25]:
#Distribuição das disciplinas
print(df['dataset'].value_counts())

dataset
Ciências da Natureza    2050
Ciências Humanas        1947
Linguagens e Códigos    1737
Matemática              1623
Name: count, dtype: int64


In [11]:
#Análise de nulos
print(df.isnull().sum())

question       0
true_answer    0
question_id    0
dataset        0
split          0
dtype: int64


In [26]:
# 1. Estrutura do dataset e informação dos dados
print("=== VISÃO GERAL DO DATASET ===")
print(f"Formato: {df.shape}")
print(f"Uso de memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nTipos de colunas:")
print(df.dtypes.value_counts())
print("\nPrimeiras linhas:")
df.head()

=== VISÃO GERAL DO DATASET ===
Formato: (7357, 5)
Uso de memória: 13.32 MB

Tipos de colunas:
object    5
Name: count, dtype: int64

Primeiras linhas:


,question,true_answer,question_id,dataset,split
0,TEXTO DE REFERÊNCIA: Nos desfiles das escolas ...,36 m²,512bec889180958c6b2e1c395931312ecc916a2039930d...,Matemática,train
1,TEXTO DE REFERÊNCIA: A companhia de energia el...,V(x) = 3x – 288,15da8d828a79fd53c164ac141d0ff2da49b16ade1618ae...,Matemática,train
2,TEXTO DE REFERÊNCIA: A forma mais simples e an...,hidratação do eteno,511f342e3329057bc6c74ff0f0c36dae381f437c3d5350...,Ciências da Natureza,train
3,TEXTO DE REFERÊNCIA: Cingapura é universalment...,projeção no mercado global,cd526d0e6d0406f30dea65939e5d2e14efebffaf28b334...,Ciências Humanas,train
4,Texto de referência: Invocação de São Mateus (...,"Estilo religioso e dramático, com clara demons...",8f97982035c3c9f4444974669aed8466b9b96a6c4ebf2d...,Linguagens e Códigos,train


In [28]:
# 2. QUALIDADE DOS DADOS E ANÁLISE DE VALORES ÚNICOS
print("=== QUALIDADE DOS DADOS ===")
print("Valores ausentes por coluna:")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Contagem dos valores ausentes': missing_data,
    'Valores ausentes %': missing_percent
}).sort_values('Valores ausentes %', ascending=False)
print(missing_df[missing_df['Contagem dos valores ausentes'] > 0])

print("\nLinhas duplicadas:", df.duplicated().sum())
print("Valores únicos por coluna:")
for col in df.columns:
    unique_count = df[col].nunique()
    print(f"{col}: {unique_count} valores únicos ({unique_count/len(df)*100:.1f}%)")

=== QUALIDADE DOS DADOS ===
Valores ausentes por coluna:
Empty DataFrame
Columns: [Contagem dos valores ausentes, Valores ausentes %]
Index: []

Linhas duplicadas: 0
Valores únicos por coluna:
question: 7357 valores únicos (100.0%)
true_answer: 6678 valores únicos (90.8%)
question_id: 7357 valores únicos (100.0%)
dataset: 4 valores únicos (0.1%)
split: 3 valores únicos (0.0%)


In [29]:
# 3. Análise da variável alvo
print("=== ANÁLISE DA VARIÁVEL ALVO ===")

# Identificando colunas que podem ser a variável alvo
potential_targets = []
for col in df.columns:
    if any(keyword in col.lower() for keyword in ['answer', 'correct', 'target', 'label', 'score', 'result']):
        potential_targets.append(col)

if potential_targets:
    print("Colunas potenciais encontradas:", potential_targets)
    for target in potential_targets:
        print(f"\nDistribuição de {target}:")
        print(df[target].value_counts().sort_index())
        if df[target].dtype in ['object', 'category']:
            print(f"Distribuição de classes:")
            print(df[target].value_counts(normalize=True).round(3))
else:
    print("Nenhuma coluna alvo óbvia encontrada. Mostrando todas as colunas:")
    print(df.columns.tolist())

=== ANÁLISE DA VARIÁVEL ALVO ===
Colunas potenciais encontradas: ['true_answer']

Distribuição de true_answer:
true_answer
"Sou aquela velhinha" e "Fiz uma tatuagem"                                                                   1
((A – B) ∩ C) ∩ ((A – D) ∪ E)                                                                                1
(-1, 3)                                                                                                      1
(-2, 1)                                                                                                      1
(1, 2)                                                                                                       1
                                                                                                            ..
“Tu é labirinto” e “Me perdi”                                                                                1
“[…] o longa entrega uma fusão de real e fictício que por vezes parece ter medo de ser mais crítico 

In [30]:
# 4. Análise de features textuais
print("=== ANÁLISE DE FEATURES TEXTUAIS ===")

text_columns = df.select_dtypes(include=['object']).columns
for col in text_columns:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  - Comprimento médio: {df[col].str.len().mean():.1f} caracteres")
        print(f"  - Comprimento mínimo: {df[col].str.len().min()}")
        print(f"  - Comprimento máximo: {df[col].str.len().max()}")
        print(f"  - Valores únicos: {df[col].nunique()}")

        # Valores de exemplo
        print(f"  - Valores de exemplo:")
        for i, sample in enumerate(df[col].dropna().head(2)):
            print(f"    [{i+1}] {str(sample)[:100]}{'...' if len(str(sample)) > 100 else ''}")

# Contagem de palavras
import re
def count_words(text):
    if pd.isna(text):
        return 0
    return len(re.findall(r'\b\w+\b', str(text)))

for col in text_columns:
    if col in df.columns:
        word_counts = df[col].apply(count_words)
        print(f"\n{col} - Contagem de palavras:")
        print(f"  Média de palavras: {word_counts.mean():.1f}")
        print(f"  Mínimo de palavras: {word_counts.min()}")
        print(f"  Máximo de palavras: {word_counts.max()}")

=== ANÁLISE DE FEATURES TEXTUAIS ===

question:
  - Comprimento médio: 795.9 caracteres
  - Comprimento mínimo: 42
  - Comprimento máximo: 5889
  - Valores únicos: 7357
  - Valores de exemplo:
    [1] TEXTO DE REFERÊNCIA: Nos desfiles das escolas de samba do Rio de Janeiro, é cada vez mais comum obse...
    [2] TEXTO DE REFERÊNCIA: A companhia de energia elétrica de uma cidade realiza a cobrança do forneciment...

true_answer:
  - Comprimento médio: 33.1 caracteres
  - Comprimento mínimo: 1
  - Comprimento máximo: 241
  - Valores únicos: 6678
  - Valores de exemplo:
    [1] 36 m²
    [2] V(x) = 3x – 288

question_id:
  - Comprimento médio: 128.0 caracteres
  - Comprimento mínimo: 128
  - Comprimento máximo: 128
  - Valores únicos: 7357
  - Valores de exemplo:
    [1] 512bec889180958c6b2e1c395931312ecc916a2039930da113eb5ed196ac010dcb7fe06e473cd64b056438961bf09bfe7b46...
    [2] 15da8d828a79fd53c164ac141d0ff2da49b16ade1618ae381b46765290af4f8226c5439dbf8beadf9923f3d6ccf68b21a248...

datas

In [31]:
# 5. Análise de features numéricas
print("=== ANÁLISE DE FEATURES NUMÉRICAS ===")

numerical_columns = df.select_dtypes(include=[np.number]).columns
if len(numerical_columns) > 0:
    print("Estatísticas das colunas numéricas:")
    print(df[numerical_columns].describe())
    
    # Checando outliers usando o método IQR
    print("\nDetecção de outliers (método IQR):")
    for col in numerical_columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df)*100:.1f}%)")
else:
    print("Sem colunas numéricas encontradas.")

=== ANÁLISE DE FEATURES NUMÉRICAS ===
Sem colunas numéricas encontradas.


In [32]:
# 6. Análise de correlação
print("=== ANÁLISE DE CORRELAÇÃO ===")

if len(numerical_columns) > 1:
    correlation_matrix = df[numerical_columns].corr()
    print("Matriz de correlação:")
    print(correlation_matrix)
    
    # Encontrar features altamente correlacionadas
    print("\nPares de features altamente correlacionadas (|correlação| > 0.7):")
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_val = correlation_matrix.iloc[i, j]
            if abs(corr_val) > 0.7:
                print(f"{correlation_matrix.columns[i]} - {correlation_matrix.columns[j]}: {corr_val:.3f}")

# Cria um mapa de calor de correlação se tivermos dados numéricos
if len(numerical_columns) > 1:
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
    plt.title('Mapa de Calor de Correlação entre Features')
    plt.tight_layout()
    plt.show()

=== ANÁLISE DE CORRELAÇÃO ===


In [33]:
# 7. Desbalanceamento de classes
print("=== ANÁLISE DE DESBALANCEAMENTO DE CLASSES ===")

# Verificando desbalanceamento de classes nas colunas categóricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
for col in categorical_columns:
    if df[col].nunique() < 20:  # Apenas analisar colunas com número razoável de categorias
        print(f"\n{col} distribuição:")
        value_counts = df[col].value_counts()
        value_props = df[col].value_counts(normalize=True)
        
        for val in value_counts.index:
            print(f"  {val}: {value_counts[val]} ({value_props[val]:.1%})")

        # Calcular a razão de desbalanceamento
        max_class = value_counts.max()
        min_class = value_counts.min()
        imbalance_ratio = max_class / min_class if min_class > 0 else float('inf')
        print(f"  Razão de desbalanceamento: {imbalance_ratio:.2f}:1")

        if imbalance_ratio > 5:
            print(f"Alerta: Desbalanceamento de classes detectado!")

=== ANÁLISE DE DESBALANCEAMENTO DE CLASSES ===

dataset distribuição:
  Ciências da Natureza: 2050 (27.9%)
  Ciências Humanas: 1947 (26.5%)
  Linguagens e Códigos: 1737 (23.6%)
  Matemática: 1623 (22.1%)
  Razão de desbalanceamento: 1.26:1

split distribuição:
  train: 5885 (80.0%)
  test: 737 (10.0%)
  valid: 735 (10.0%)
  Razão de desbalanceamento: 8.01:1
Alerta: Desbalanceamento de classes detectado!


In [35]:
# 8. Detecção de Data Leakage
print("=== DETECÇÃO DE DATA LEAKAGE ===")

# Verificar features que podem causar data leakage
suspicious_keywords = ['id', 'key', 'index', 'uuid', 'timestamp', 'created', 'updated', 'modified']
potential_leakage = []

for col in df.columns:
    col_lower = col.lower()
    
    # Verificar nomes de colunas suspeitas
    if any(keyword in col_lower for keyword in suspicious_keywords):
        potential_leakage.append((col, "Nome de coluna suspeito"))

    # Verificar colunas com muitos valores únicos (potenciais IDs)
    if df[col].nunique() == len(df):
        potential_leakage.append((col, "Identificador único (100% valores únicos)"))

    # Verificar colunas com alta cardinalidade
    elif df[col].nunique() / len(df) > 0.95:
        potential_leakage.append((col, f"Alta cardinalidade ({df[col].nunique()/len(df):.1%} únicos)"))

if potential_leakage:
    print("Fontes potenciais de data leakage encontradas:")
    for col, reason in potential_leakage:
        print(f"{col}: {reason}")
else:
    print("Nenhuma fonte óbvia de data leakage detectada.")

=== DETECÇÃO DE DATA LEAKAGE ===
Fontes potenciais de data leakage encontradas:
question: Identificador único (100% valores únicos)
question_id: Nome de coluna suspeito
question_id: Identificador único (100% valores únicos)


In [36]:
# 9. Amostragem e Estratégia de Subconjunto
print("=== AMOSTRAGEM E ESTRATÉGIA DE SUBCONJUNTO ===")

print(f"Tamanho do conjunto de dados: {len(df):,} linhas")
print(f"Uso de memória: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Complexidade computacional estimada para diferentes tamanhos de amostra
sample_sizes = [1000, 5000, 10000, 25000, 50000, 100000]
for size in sample_sizes:
    if size <= len(df):
        percentage = (size / len(df)) * 100
        print(f"Amostra de {size:,} linhas = {percentage:.1f}% do conjunto de dados")

# Recomendar estratégia de amostragem com base nas características do conjunto de dados
print("\nRECOMENDAÇÕES DE AMOSTRAGEM:")
if len(df) > 100000:
    print("• Considere amostragem estratificada para manter as distribuições de classes")
    print("• Comece com 10-20% da amostra para o desenvolvimento inicial do modelo")
    print("• Use amostragem progressiva: 1K → 10K → 50K → conjunto de dados completo")
elif len(df) > 10000:
    print("• Conjunto de dados de tamanho médio - considere 20-50% da amostra para prototipagem")
    print("• Conjunto de dados completo gerenciável para a maioria dos algoritmos")
else:
    print("• Conjunto de dados pequeno - use o conjunto de dados completo ou validação cruzada cuidadosa")
    print("• Tenha cuidado com o overfitting com dados limitados")

# Verificar se o conjunto de dados precisa de considerações especiais de amostragem
text_heavy = any(df[col].dtype == 'object' and df[col].str.len().mean() > 500
                for col in df.select_dtypes(include=['object']).columns)
if text_heavy:
    print("• Conjunto de dados com muito texto detectado - considere amostragem baseada em comprimento de texto")

high_cardinality = any(df[col].nunique() > len(df) * 0.1 
                      for col in df.columns)
if high_cardinality:
    print("• Recursos de alta cardinalidade detectados - garanta representação nas amostras")

=== AMOSTRAGEM E ESTRATÉGIA DE SUBCONJUNTO ===
Tamanho do conjunto de dados: 7,357 linhas
Uso de memória: 19.5 MB
Amostra de 1,000 linhas = 13.6% do conjunto de dados
Amostra de 5,000 linhas = 68.0% do conjunto de dados

RECOMENDAÇÕES DE AMOSTRAGEM:
• Conjunto de dados pequeno - use o conjunto de dados completo ou validação cruzada cuidadosa
• Tenha cuidado com o overfitting com dados limitados
• Conjunto de dados com muito texto detectado - considere amostragem baseada em comprimento de texto
• Recursos de alta cardinalidade detectados - garanta representação nas amostras


In [37]:
# 10. Insights para Treinamento de Modelos
print("=== INSIGHTS PARA TREINAMENTO DE MODELOS ===")

# Recomendações de engenharia de features
print("RECOMENDAÇÕES DE ENGENHARIA DE FEATURES:")

text_cols = df.select_dtypes(include=['object']).columns
if len(text_cols) > 0:
    print("• Recursos de texto detectados - considere:")
    print("  - Vetorização TF-IDF")
    print("  - Word embeddings (Word2Vec, GloVe, BERT)")
    print("  - Comprimento do texto como recurso numérico")
    print("  - Detecção de idioma se multilíngue")

if len(numerical_columns) > 0:
    print("• Recursos numéricos - considere:")
    print("  - Normalização/Padronização")
    print("  - Recursos polinomiais para relações não lineares")
    print("  - Binning para variáveis contínuas")

categorical_cols = df.select_dtypes(include=['category']).columns
if len(categorical_cols) > 0:
    print("• Recursos categóricos - considere:")
    print("  - One-hot encoding para baixa cardinalidade")
    print("  - Target encoding para alta cardinalidade")
    print("  - Embeddings para muito alta cardinalidade")

# Recomendações de seleção de modelos
print("\nRECOMENDAÇÕES DE SELEÇÃO DE MODELOS:")
print("Com base nas características do conjunto de dados:")

if len(df) < 1000:
    print("• Conjunto de dados pequeno → Modelos simples (Regressão Logística, SVM, Random Forest)")
elif len(df) < 100000:
    print("• Conjunto de dados médio → Métodos de ensemble (XGBoost, LightGBM, Random Forest)")
else:
    print("• Conjunto de dados grande → Deep learning viável (Redes Neurais, Transformers)")

if text_heavy:
    print("• Conjunto de dados com muito texto → Modelos Transformer (BERT, RoBERTa) ou arquiteturas CNN/RNN")

# Estratégia de validação cruzada
print("\nESTRATÉGIA DE VALIDAÇÃO CRUZADA:")
if len(df) < 1000:
    print("• Use stratified k-fold (k=5 ou k=10)")
elif len(df) < 10000:
    print("• Use stratified k-fold (k=5) ou divisão treino/validação/teste")
else:
    print("• Divisão simples treino/validação/teste (70/15/15 ou 80/10/10)")

print("\nMÉTRICAS CHAVE PARA ACOMPANHAR:")
print("• Curvas de perda de treinamento/validação")
print("• Precisão, Recall, F1-score por classe")
print("• Matriz de confusão para análise de erros")
print("• Importância de recursos/pesos de atenção")
print("• Tempo de treinamento e velocidade de inferência")

=== INSIGHTS PARA TREINAMENTO DE MODELOS ===
RECOMENDAÇÕES DE ENGENHARIA DE FEATURES:
• Recursos de texto detectados - considere:
  - Vetorização TF-IDF
  - Word embeddings (Word2Vec, GloVe, BERT)
  - Comprimento do texto como recurso numérico
  - Detecção de idioma se multilíngue

RECOMENDAÇÕES DE SELEÇÃO DE MODELOS:
Com base nas características do conjunto de dados:
• Conjunto de dados médio → Métodos de ensemble (XGBoost, LightGBM, Random Forest)
• Conjunto de dados com muito texto → Modelos Transformer (BERT, RoBERTa) ou arquiteturas CNN/RNN

ESTRATÉGIA DE VALIDAÇÃO CRUZADA:
• Use stratified k-fold (k=5) ou divisão treino/validação/teste

MÉTRICAS CHAVE PARA ACOMPANHAR:
• Curvas de perda de treinamento/validação
• Precisão, Recall, F1-score por classe
• Matriz de confusão para análise de erros
• Importância de recursos/pesos de atenção
• Tempo de treinamento e velocidade de inferência
